In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install PyPDF2 pdfplumber nltk --quiet
import os
import pdfplumber
import re
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.4 MB/s eta 0:00:00:00:01


In [3]:


# Extract text from a single page with whitespace handling
def extract_text_with_whitespace(page):
    chars = page.chars
    chars.sort(key=lambda c: (c["top"], c["x0"]))

    text = ""
    previous_x1 = None

    for char in chars:
        if previous_x1 is not None:
            gap = char["x0"] - previous_x1
            if gap > 1:  # Adjust this value based on the spacing in your PDFs
                text += " "
        text += char["text"]
        previous_x1 = char["x1"]

    return text




In [4]:
# Extract text starting from the "Introduction" section
def extract_text_from_introduction(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        found_intro = False
        extracted_text = ""
        for page in pdf.pages:
            text = extract_text_with_whitespace(page)
            extracted_text += text
        return extracted_text


In [5]:
# Extract text starting from the "Introduction" section
def extract_text_from_introduction(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        found_intro = False
        extracted_text = ""
        for page in pdf.pages:
            text = extract_text_with_whitespace(page)
            extracted_text += text
        return extracted_text

# # Process all PDFs in a directory
# def process_pdf_directory(directory, category):
#     data = []
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             if file.endswith('.pdf'):
#                 file_path = os.path.join(root, file)
#                 try:
#                     text = extract_text_from_introduction(file_path)
#                     # Extract conference type if it's a publishable file
#                     conference_type = None
#                     if category == 1:  # Publishable category
#                         # Assuming conference type is part of the folder name
#                         conference_type = os.path.basename(root)
#                     file_name = os.path.basename(file_path)
#                     identifier = os.path.splitext(file_name)[0]
#                     data.append({
#                         "Paper ID": identifier,
#                         "text": text,
#                         "category": category,
#                         "Conference Type": conference_type
#                     })
#                 except Exception as e:
#                     print(f"Error processing {file_path}: {e}")
#     return data

# # Define directories for publishable and non-publishable PDFs
# non_publishable_path = 'Reference/Non-Publishable'
# publishable_path = 'Reference/Publishable'

# # Process each category
# non_publishable_data = process_pdf_directory(non_publishable_path, category=0)
# publishable_data = process_pdf_directory(publishable_path, category=1)

# # Combine the results
# all_data = non_publishable_data + publishable_data

# # Create a DataFrame
# df_train = pd.DataFrame(all_data)

# df_train




In [6]:
path = '/kaggle/input/attention-is-all-you-need/Attention.pdf'
text = extract_text_from_introduction(path)
text

'Provided proper attribution is provided, Google hereby grants permission toreproduce the tables and figures in this paper solely for use in journalistic orscholarly works.Attention Is All You Need32 ∗ ∗ ∗ ∗0 Ashish Vaswani Noam Shazeer Niki Parmar Jakob Uszkoreit2 Google Brain Google Brain Google Research Google Research  avaswani@google.com noam@google.com nikip@google.com usz@google.comguA ∗ ∗ † ∗Llion Jones Aidan N. Gomez Łukasz Kaiser  Google Research University of Toronto Google Brain2 lukaszkaiser@google.comllion@google.com aidan@cs.toronto.edu  ]L ∗ ‡Illia PolosukhinC illia.polosukhin@gmail.com.sc[   Abstract7v2 The dominant sequence transduction models are based on complex recurrent or6 convolutional neural networks that include an encoder and a decoder. The best7 performing models also connect the encoder and decoder through an attention3 mechanism. We propose a new simple network architecture, the Transformer,0 based solely on attention mechanisms, dispensing with recurrence

In [7]:

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = re.sub(r'[^A-Za-z0-9\s.,;:!?]', '', text)  # Remove special characters
    return text

In [8]:
raw_text = extract_text_from_introduction(path)
cleaned_text = clean_text(raw_text)
print(cleaned_text)

Provided proper attribution is provided, Google hereby grants permission toreproduce the tables and figures in this paper solely for use in journalistic orscholarly works.Attention Is All You Need32    0 Ashish Vaswani Noam Shazeer Niki Parmar Jakob Uszkoreit2 Google Brain Google Brain Google Research Google Research  avaswanigoogle.com noamgoogle.com nikipgoogle.com uszgoogle.comguA    Llion Jones Aidan N. Gomez ukasz Kaiser  Google Research University of Toronto Google Brain2 lukaszkaisergoogle.comlliongoogle.com aidancs.toronto.edu  L  Illia PolosukhinC illia.polosukhingmail.com.sc   Abstract7v2 The dominant sequence transduction models are based on complex recurrent or6 convolutional neural networks that include an encoder and a decoder. The best7 performing models also connect the encoder and decoder through an attention3 mechanism. We propose a new simple network architecture, the Transformer,0 based solely on attention mechanisms, dispensing with recurrence and convolutions.6 en